In [ ]:
!unzip /content/drive/MyDrive/PBL/flooding/dataset/flood_dataset5.zip -d ../content/flood_dataset5

In [ ]:
# 이미지 처리와 시각 처리를 위한 도구
import numpy as np # 다차원 배열과 행렬 연산
import os # 운영체제
import cv2 # OpenCV 라이브러리
import matplotlib.pyplot as plt # Matplotlib 라이브러리, 데이터 시각화 라이브러리, 그래프 및 도표 생성
%matplotlib inline

In [ ]:
flood_images_path = r"../content/flood_dataset5/flood4"
non_flood_images_path = r"../content/flood_dataset5/non_flood4"

In [ ]:
# 침수 이미지 표시
for img in os.listdir(flood_images_path):# 화재 데이터셋 가져오기
    img_array = cv2.imread(os.path.join(flood_images_path,img)) # 화재 이미지 opencv로 읽어서 img_array에 저장
    plt.imshow(img_array) # img_array에 저장된 이미지 표시
    plt.title(img)  # 이미지 위에 파일 이름을 출력
    plt.show() # 화면에 이미지 출력
    break # 맨 처음 이미지 출력하고 종료, 이거 제거하면 여러 이미지 출력 가능

In [ ]:
# 비침수 이미지 표시
for img in os.listdir(non_flood_images_path):
    img_array = cv2.imread(os.path.join(non_flood_images_path,img))
    plt.imshow(img_array)
    plt.show()
    break

In [ ]:
# img_array의 형태 표시, 이미지 크기와 색상 채널 수 표시
img_array.shape # (높이, 너비, 채널) 튜플 반환, 채널이 3이면 컬러, 1이면 흑백

In [ ]:
# 이미지 데이터 전처리
data = [] # 이미지 데이터와 클래스 레이블을 저장할 리스트 선언
IMG_SIZE = 32 # 모든 이미지 32x32 픽셀로 조정

def get_data(file_path,class_num, data): # 이미지 데이터 data에 저장
    for img in os.listdir(file_path):
        try:
            img_array = cv2.imread(os.path.join(file_path,img)) # 이미지 파일 경로 저장, 파일을 img_array에 저장
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) # 이미지 크기 리사이징
            data.append([new_array, class_num]) # 이미지와 클래스 레이블 엮서서 data 리스트에 추가
        except:
            pass # 예외 무시하고 처리
get_data(flood_images_path,1,data) # 1이면 침수 이미지
get_data(non_flood_images_path,0,data) # 0이면 비침수 이미지

In [ ]:
# data 리스트의 개수 반환
len(data)

In [ ]:
# 텐서플로우 모델 정의
import tensorflow as tf # 텐서플로우 라이브러리 가져오기
from tensorflow import keras # 텐서플로우의 케라스 모듈 가져오기
from tensorflow.keras import layers, models # 케라스에서 서브 모듈인 layers, models 가져오기
from keras.layers import Dense, Activation, Flatten, Dropout # 순서대로 연결레이어, 활성화함수, 배열평탄화, 과적합방지

In [ ]:
# 데이터 증강으로 이미지 데이터셋 확장 (텐서플로우 이용)
data_augmentation1 = tf.keras.Sequential([  # 데이터 증강 파이프라인 정의 1
  layers.experimental.preprocessing.RandomZoom(0.5), # 이미지의 0~50% 범위에서 무작위로 줌 적용
])

data_augmentation2 = tf.keras.Sequential([ # 데이터 증강 파이프라인 정의 2
  layers.experimental.preprocessing.RandomContrast(0.2) # 이미지의 대비를 무작위로 조정
])

def apply_augmentation(data): # 데이터 증강 함수
    flood_count = sum(1 for _, label in data if label == 1) # 침수 이미지의 개수 계산
    non_flood_count = sum(1 for _, label in data if label == 0) # 비침수 이미지의 개수 계산

    while non_flood_count < flood_count: # 비침수 이미지 수가 침수 이미지 수와 같아질 때까지 반복
        for i in range(len(data)):
            if data[i][1] == 0: # 데이터 레이블이 0, 즉 비침수 이미지에만 데이터 증강 수행
                data.append([data_augmentation1(data[i][0]), 0]) # 변형된 추가 데이터 생성
                non_flood_count += 1 # 비침수 이미지 개수 증가
                if non_flood_count >= flood_count: # 비침수 이미지 수가 침수 이미지 수와 같거나 크면 반복 종료
                    break

apply_augmentation(data) # 변형해서 확장된 데이터셋을 원래 데이터와 함께 data 리스트에 추가

# 침수 이미지 수와 비침수 이미지 수 출력
flood_count = sum(1 for _, label in data if label == 1)
non_flood_count = sum(1 for _, label in data if label == 0)
print("침수 이미지 수:", flood_count)
print("비침수 이미지 수:", non_flood_count)

len(data) # 전체 데이터셋 크기 확인

In [ ]:
# 데이터 증강으로 이미지 데이터셋 확장 (텐서플로우 이용)
data_augmentation1 = tf.keras.Sequential([  # 데이터 증강 파이프라인 정의 1
  layers.experimental.preprocessing.RandomZoom(0.5), # 이미지의 0~50% 범위에서 무작위로 줌 적용
])

data_augmentation2 = tf.keras.Sequential([ # 데이터 증강 파이프라인 정의 2
  layers.experimental.preprocessing.RandomContrast(0.2) # 이미지의 대비를 무작위로 조정
])

def apply_augmentation(data): # 데이터 증강 함수
    for i in range(len(data)):
        if data[i][1] == 0 or data[i][1] == 1:
            data.append([data_augmentation1(data[i][0]), data[i][1]])  # 변형된 추가 데이터 생성
            data.append([data_augmentation2(data[i][0]), data[i][1]])  # 변형된 추가 데이터 생성
apply_augmentation(data) # 변형해서 확장된 데이터셋을 원래 데이터와 함께 data 리스트에 추가
len(data) # 전체 데이터셋 크기 확인

In [ ]:
# 데이터셋 섞기
import random
random.shuffle(data) # 데이터셋 랜덤으로 섞기 (모델 학습 시에 데이터 순서에 영향 안 받게 함)

In [ ]:
# 이미지 데이터와 클래스 레이블 분리 (데이터셋 준비 과정)
X = [] # 이미지 데이터 저장 리스트
y = [] # 이미지의 클래스 레이블 저장 리스트

for i in data:
    X.append(i[0]) # i[0]이 데이터의 첫 번째 요소로 이미지 데이터
    y.append(i[1]) # i[1]이 데이터의 두 번재 요소로 클래스 레이블

In [ ]:
# 데이터셋에서 이미지, 클래스 레이블 확인
X[0],y[0]

In [ ]:
# 클래스 레이블 데이터 시각화
plt.hist(y)

In [ ]:
# 이미지 데이터 형태 조정, X랑 y 리스트 각각 배열 변환
X = np.array(X).reshape(-1,IMG_SIZE, IMG_SIZE,3) # 4차원 배열로 변환
y = np.array(y) # 1차원 배열로 변환

In [ ]:
# 배열 X와 y 형태 변환
X.shape, y.shape
# X는 (데이터 개수, 높이, 너비, 채널 수), y는 (클래스 레이블 개수, )

In [ ]:
# 데이터셋을 학습 용과 테스트 용으로 나누기
from sklearn.model_selection import train_test_split

# 데이터 X와 레이블 y를 학습 셋과 테스트 셋으로 나눔, 테스트 셋은 20%
# 테스트 셋을 검증 셋과 최종 테스트 셋으로 나눔
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2)
X_test,X_val,y_test,y_val = train_test_split(X_test,y_test,test_size=0.5)

In [ ]:
# 데이터셋 형태 출력
print(X_train.shape) # 학습 세트의 형태 출력 (데이터 수, 이미지 높이, 너비, 채널 수)
print(X_test.shape) # 테스트 세트의 형태 출력
print(X_val.shape) # 검증 세트의 형태 출력

In [ ]:
# 텐서플로우로 모델 정의
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from keras.layers import Dense, Activation, Flatten, Dropout

In [ ]:
# 모델 분류
model = models.Sequential([ # 순차적으로 레이어를 쌓아가는 Keras의 모델
  # 첫 번째 합성곱 레이어 (Conv2D), 입력 이미지의 특징 추출
  # 'padding'은 'same'으로 설정되어 입력과 출력 크기 동일하게 유지
  layers.Conv2D(32, kernel_size = (3,3), padding='same', activation='relu', input_shape=(IMG_SIZE,IMG_SIZE,3)),
  # 첫 번째 맥스 풀링 레이어 (MaxPooling2D)
  # 공간 크기 감소 및 계산 비용을 위해 사용
  layers.MaxPooling2D(),

  layers.Conv2D(64, kernel_size = (3,3), padding='same', activation='relu'),
  layers.MaxPooling2D(),

  layers.Dropout(0.5), # 드롭 아웃 레이어

  layers.Conv2D(64, kernel_size = (3,3), padding='same', activation='relu'),
  layers.MaxPooling2D(),

  layers.Conv2D(64, kernel_size = (3,3), padding='same', activation='relu'),
  layers.MaxPooling2D(),

  # 최종 분류 수행
  layers.Flatten(), # 다차원 배열을 1차원 배열로 변환
  layers.Dense(60, activation='relu'),  # 첫 번째 Fully Connected 레이어 (Dense), 모든 뉴런이 이전 레이어의 모든 뉴런과 연결
  layers.Dropout(0.5), # 드롭 아웃 레이어
  layers.Dense(1, activation='sigmoid') # 두 번째 Fully Connected 레이어 (Dense), 이진 분류 문제에서 출력을 생성
])

In [ ]:
# 모델 구조 출력
model.summary()

In [ ]:
# 모델 컴파일
from tensorflow.keras.optimizers import Adam

custom_optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=custom_optimizer, metrics=['accuracy'])

In [ ]:
# 모델 훈련, epochs는 반복하는 학습 횟수 에폭
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=40) # train은 훈련 데이터, validation은 검증 데이터

In [ ]:
# 정확도 함수
plt.figure(0) # 그래프 창 생성
plt.plot(history.history['accuracy'], label='training accuracy') # 학습 정확도 라벨 붙여서 표시
plt.plot(history.history['val_accuracy'], label='val accuracy') # 검증 정확도 라벨 붙여서 표시
plt.title("Accuracy") # 그래프 제목 설정
plt.xlabel('epochs') # x축 레이블을 학습 수로 설정
plt.ylabel('accuracy') # y축 레이블을 정확도로 설정
plt.legend() # 그래프에 범례 추가
plt.show() # 화면에 그래프 표시

# 손실 함수 (오차 함수)
plt.figure(1) # 그래프 창 생성
plt.plot(history.history['loss'], label='training loss') # 손실
plt.plot(history.history['val_loss'], label='val loss') # 검증 손실
plt.title('Loss') # 그래프 제목 설정
plt.xlabel('epochs') # x축 레이블을 학습 수로 설정
plt.ylabel('loss') # y축 레이블을 손실로 설정
plt.legend() # 그래프에 범례 추가
plt.show() # 화면에 그래프 표시